----
<div style="display: flex; align-items: center;">
    <img src="https://developers.lseg.com/content/dam/devportal/icons/logo/lseg-logo.svg" width="20%" style="vertical-align: top;">
    <h1 style="margin-left: 20px;">Data Library for Python</h1>
</div>

----

## Delivery - OMM Stream - Market Price data via callback

This notebook demonstrates how to use the OMM Stream interface to request streaming Quote and Trade data

## Set the location of the configuration file
For ease of use, you can set various initialization parameters of the RD Library in the **_lseg-data.config.json_** configuration file - as described in the Quick Start -> Sessions example.

### One config file for the tutorials
As these tutorial Notebooks are categorised into sub-folders and to avoid the need for multiple config files, we will use the _LD_LIB_CONFIG_PATH_ environment variable to point to a single instance of the config file in the top-level ***Configuration*** folder.

Before proceeding, please **ensure you have entered your credentials** into the config file in the ***Configuration*** folder.

In [1]:
import os
os.environ["LD_LIB_CONFIG_PATH"] = "../../../Configuration"

In [1]:
from lseg.data.delivery import omm_stream
import lseg.data as ld
import datetime
import json

## Open the default session

To open the default session ensure you have a '*lseg-data.config.json*' in the ***Configuration*** directory, populated with your credentials and specified a 'default' session in the config file    


In [2]:
ld.open_session()

<lseg.data.session.Definition object at 0x1b571900410 {name='workspace'}>

## Define a function to display events

In [3]:
# Callback function to display data or status events
def display_event(eventType, event):
    currentTime = datetime.datetime.now().time()
    print("----------------------------------------------------------")
    print(">>> {} event received at {}".format(eventType, currentTime))
    print(json.dumps(event, indent=2))
    return

## Create an Item Stream and register the event callbacks

In [4]:
stream = omm_stream.Definition(
    name="EUR=", 
    fields=['BID', 'ASK','OPEN_PRC','HST_CLOSE','TIMACT'], 
    domain='MarketPrice').get_stream()

# Refresh - the first full image we get back from the server
stream.on_refresh(lambda event, stream  : display_event("Refresh", event))
# Update - as and when field values change, we receive updates from the server
stream.on_update(lambda event, stream : display_event("Update", event))
# Status - if data goes stale or item closes, we get a status message
stream.on_status(lambda event, stream : display_event("Status", event))
# Other errors
stream.on_error(lambda event, stream : display_event("Error", event))

## Open the OMM Stream
The **open()** call to open the OMM Stream is a synchronous one. This means the first event (either via on_refresh(), on_status() or on_error()) can occur before the open() method returns.   
However, if we use the **open_async()** asynchronous method instead, the first event callback will be after  open_async() returns.

In [5]:
# Send request to server and open stream
stream.open()
# We should receive the initial Refresh for the current field values
# followed by updates for the fields as and when they occur

----------------------------------------------------------
>>> Refresh event received at 18:41:45.690782
{
  "Fields": {
    "ASK": 1.0748,
    "BID": 1.0747,
    "HST_CLOSE": 1.0738,
    "OPEN_PRC": 1.0739,
    "TIMACT": "16:41:00"
  },
  "State": {
    "Data": "Ok",
    "Stream": "Open",
    "Code": "None",
    "Text": "All is well"
  },
  "Qos": {
    "Rate": "TickByTick",
    "Timeliness": "Realtime"
  },
  "Type": "Refresh",
  "SeqNumber": 9566,
  "ID": 5,
  "Key": {
    "Name": "EUR=",
    "NameType": "Ric",
    "Service": "Q"
  }
}


<OpenState.Opened: 'Opened'>

## Close Stream

In [6]:
stream.close()

<OpenState.Closed: 'Closed'>

## Close the session

In [7]:
ld.close_session()